
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# LAB - AutoML

Welcome to the AutoML Lab! In this lab, you will explore the capabilities of AutoML using the Databricks AutoMl UI and AutoML API. 


**Lab Outline:**

In this lab, you will need to complete the following tasks;

* **Task 1 :** Load data set.

* **Task 2 :** Create a classification experiment using the AutoML UI.

* **Task 3 :** Create a classification experiment with the AutoML API using a feature table.

* **Task 4 :** Retrieve the best run and show the model URI.

* **Task 5 :** Import the notebook for a run.



## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.
Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.
1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:
   - In the drop-down, select **More**.
   - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.
  
**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:
1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.
1. Find the triangle icon to the right of your compute cluster name and click it.
1. Wait a few minutes for the cluster to start.
1. Once the cluster is running, complete the steps above to select your cluster.

## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **16.3.x-cpu-ml-scala2.12**


## Classroom Setup

Before starting the demo, run the provided classroom setup script. This script will define configuration variables necessary for the demo. Execute the following cell:

In [0]:
%run ../Includes/Classroom-Setup-3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/569.1 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━ 358.4/569.1 kB 10.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: databricks-sdk
    Found existing installation: databricks-sdk 0.30.0
    Not uninstalling databricks-sdk at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-4987ddcc-e27b-4745-bffe-98519f9550af
    Can't uninstall 'databricks-sdk'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


2025/07/02 17:02:31 INFO databricks.ml_features._compute_client._compute_client: Setting columns ['ID'] of table 'dbacademy.labuser10773581_1751474919.bank_loan_features' to NOT NULL.
2025/07/02 17:02:35 INFO databricks.ml_features._compute_client._compute_client: Setting Primary Keys constraint ['ID'] on table 'dbacademy.labuser10773581_1751474919.bank_loan_features'.
2025/07/02 17:02:42 INFO databricks.ml_features._compute_client._compute_client: Created feature table 'dbacademy.labuser10773581_1751474919.bank_loan_features'.


**Other Conventions:**

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"User DB Location:  {DA.paths.datasets}")

Username:          labuser10773581_1751474919@vocareum.com
Catalog Name:      dbacademy
Schema Name:       labuser10773581_1751474919
Working Directory: /Volumes/dbacademy/ops/labuser10773581_1751474919@vocareum_com
User DB Location:  NestedNamespace (california_housing='/Volumes/dbacademy_california_housing/v02', cdc_diabetes='/Volumes/dbacademy_cdc_diabetes/v01', telco='/Volumes/dbacademy_telco/v01', banking='/Volumes/dbacademy_banking/v01')


## Task 1 : Load data set

Load the dataset that will be used for the AutoML experiment.

* Load and display the dataset where the table name is **`bank_loan`**.

* Load and display the feature table where the feature table is **`bank_loan_features`**.

In [0]:
loan_data = spark.sql("Select * from bank_loan")
display(loan_data)

ID,Age,ZIP_Code,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
1,25,91107,4,1.60,1,0,0,1,0,0,0
2,45,90089,3,1.50,1,0,0,1,0,0,0
3,39,94720,1,1.00,1,0,0,0,0,0,0
4,35,94112,1,2.70,2,0,0,0,0,0,0
5,35,91330,4,1.00,2,0,0,0,0,0,1
6,37,92121,4,0.40,2,155,0,0,0,1,0
7,53,91711,2,1.50,2,0,0,0,0,1,0
8,50,93943,1,0.30,3,0,0,0,0,0,1
9,35,90089,3,0.60,2,104,0,0,0,1,0
10,34,93023,1,8.90,3,0,1,0,0,0,0


In [0]:
loan_features_data = spark.sql("select * from bank_loan_features")
display (loan_features_data)

ID,Experience,Income
## Attribution,null,null
This course uses the Bank Loan Modeling Dataset from Thera Bank,null,null
## License Information,null,null
"You can find more details about the ""Bank Loan Modeling"" on this [Kaggle page](https://www.kaggle.com/datasets/itsmesunil/bank-loan-modelling).",null,null
"The ""Bank Loan Modeling Dataset"" is distributed under the [CC0: Public Domain](https://www.kaggle.com/datasets/itsmesunil/bank-loan-modelling). Please review the license terms before using the dataset.",null,null
For any questions or inquiries regarding the dataset or its usage,null,null
---,null,null
Note: This README is provided as part of the dataset attribution for educational purposes. Please ensure compliance with the dataset's licensing terms and conditions.,null,null
1,1,49
2,19,34


## Task 2: Create Classification Experiment Using AutoML UI

Follow these steps to create an AutoML experiment using the  UI:

  ***Step 1.*** Navigate to the **Experiments** section.

  ***Step 2.*** Click on **Classification**.

  ***Step 3.*** Choose a cluster for experiment execution.

  ***Step 4.*** Select the input training dataset as **`catalog > database > bank_loan`**.

  ***Step 5.*** Specify **`Personal_Loan`** as the prediction target.

  ***Step 6.*** Deselect the **`ID`**, **`ZIP_Code`** field as it's not needed as a feature.

  ***Step 7.*** Enter a name for your experiment, like `Bank_Loan_Prediction_AutoML_Experiment`.

  ***Step 8.*** In the **Advanced Configuration** section, set the **Timeout** to **5 minutes**.

  ***Step 9.*** Click on **Start AutoML**.

## Task 3: Create a Classification Experiment with the AutoML API

Utilize the AutoML API to set up and run a classification experiment. Follow these steps:

1. **Setting up the Experiment:**
   
   - **Specify the Dataset:** Specify the dataset using the Spark table name, which is **`bank_loan`**.
   
   - **Set Target Column:** Assign the target_col to the column you want to predict, which is **`Personal_Loan`**.
   
   - **Adjust Exclude Columns:** Provide a list of columns to exclude from the modeling process after reviewing the displayed dataset.
   
   - **Use features table** to be used as part of training. Feature table name: **`bank_loan_features`**.
   
   - **Set Timeout Duration:** Determine the timeout_minutes for the AutoML experiment. such as `5` minutes.   

2. **Running AutoML:**
   - Use the AutoML API to explore various machine learning models.



In [0]:
from databricks import automl
from datetime import datetime

features_table_path = f"{DA.catalog_name}.{DA.schema_name}.bank_loan_features"

# Define the feature store lookups
feauture_store_lookups = [
    {
        "table_name": features_table_path,
        "lookup_key": ["ID"]
    }
] 

summary = automl.classify(
    dataset = loan_data,
    target_col = "Personal_Loan",
    exclude_cols = ["ID", "ZIP_Code"],  # Exclude columns as needed
    timeout_minutes = 5,
    feature_store_lookups = feauture_store_lookups
)

2025/07/02 18:04:16 INFO databricks.automl.client.manager: AutoML will optimize for F1 score metric, which is tracked as val_f1_score in the MLflow experiment.
2025/07/02 18:04:17 INFO databricks.automl.client.manager: MLflow Experiment ID: 1764296177273406
2025/07/02 18:04:17 INFO databricks.automl.client.manager: MLflow Experiment: https://dbc-8b9f7bce-656b.cloud.databricks.com/?o=182135318479115#mlflow/experiments/1764296177273406
2025/07/02 18:05:39 INFO databricks.automl.client.manager: Data exploration notebook: https://dbc-8b9f7bce-656b.cloud.databricks.com/?o=182135318479115#notebook/1764296177273411
2025/07/02 18:09:52 INFO databricks.automl.client.manager: AutoML experiment completed successfully.


,Train,Validation,Test
f1_score,0.591,0.524,0.490
recall_score,0.456,0.400,0.356
roc_auc,0.918,0.867,0.888
false_negatives,153.000,57.000,67.000
false_positives,24.000,12.000,10.000
example_count,3030.000,996.000,974.000
precision_score,0.842,0.760,0.787
true_positives,128.000,38.000,37.000
precision_recall_auc,0.723,0.642,0.653
true_negatives,2725.000,889.000,860.000


## Task 4: Retrieve the best run and show the model URI

Identify the best model generated by AutoML based on a chosen metric. Retrieve information about the best run, including the model URI, to further explore and analyze the model.
 + Find the experiment id associated with your AutoML run experiment. 
 + Define a search term to filter for runs. Adjust the search term based on the desired status, such as `FINISHED` or `ACTIVE`. 
 + Specify the run view type to view only active runs or to view all runs.
 + Provide the metric you want to use for ordering  and Specify whether you want to order the runs in descending or ascending order.

In [0]:
import mlflow
from mlflow.entities import ViewType

## Find the best run ...
automl_runs_pd = mlflow.search_runs(
  experiment_ids=[summary.experiment.experiment_id],
  filter_string=f"attributes.status = 'FINISHED'",
  run_view_type=ViewType.ACTIVE_ONLY,
  order_by=["metrics.val_f1_score DESC"],
)

In [0]:
## Print information about the best trial
print(summary.best_trial)


Model: LogisticRegression(C=0.40489943114047827, penalty='l1', random_state=134702309,
           solver='saga')
Model path: dbfs:/databricks/mlflow-tracking/1764296177273406/71d17ef0abbe4a7e8ac2bc23c3ed4031/artifacts/model
Preprocessors: [('boolean', Pipeline(steps=[('cast_type',
         FunctionTransformer(func=<function <lambda> at 0x78909aec6d40>)),
        ('imputers',
         ColumnTransformer(remainder='passthrough', transformers=[])),
        ('onehot',
         OneHotEncoder(drop='first', handle_unknown='ignore'))]), ['CD_Account', 'CreditCard', 'Securities_Account', 'Online']), ('numerical', Pipeline(steps=[('converter',
         FunctionTransformer(func=<function <lambda> at 0x7890b8749ee0>)),
        ('imputers',
         ColumnTransformer(transformers=[('impute_mean',
                                          SimpleImputer(),
                                          ['CCAvg', 'Mortgage'])])),
        ('standardizer', StandardScaler())]), ['Mortgage', 'CCAvg']), ('oneho


## Task 5: Import Notebook for a Run

AutoML automatically generates the best run's notebook and makes it available for you. If you want to access to other runs' notebooks, you need to import them.

In this task, you will import the **5th run's notebook** to the **`destination_path`**. 

Show the `url` and `path` of the imported notebook.

In [0]:
destination_path = f"/Users/{DA.username}/imported_notebooks/lab.3-{datetime.now().strftime('%Y%m%d%H%M%S')}"

## Get the path and url for the generated notebook
result = automl.import_notebook(summary.trials[1].artifact_uri, destination_path)
print(result.path)
print(result.url)

/Users/labuser10773581_1751474919@vocareum.com/imported_notebooks/lab.3-20250702181346
https://dbc-8b9f7bce-656b.cloud.databricks.com/?o=182135318479115/#workspace/Users/labuser10773581_1751474919@vocareum.com/imported_notebooks/lab.3-20250702181346



## Conclusion

In this lab, you got hands-on with Databricks AutoML. You started by loading a dataset and creating a classification experiment using the AutoMl UI and AutoML API. You then learned how to summarize the best model by applying specific filters and explored the process of retrieving the best model along with its Model URI.


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>